# Multi-Agent Scenario

Provides an environment to perform auctions between different agents

In [68]:
#Libraries
import os
import random
import numpy as np
import pandas as pd
import math
from tqdm import tnrange, tqdm_notebook


from glob import glob
from IPython.display import display
import winsound

In [2]:
%%time

#Load Validation dataset & calculated pCTRs

Path = 'C://Datasets//Multiagent//calc//pCTRs'

#Importing pCTR data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*validation_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Importing Validation data.
os.chdir('c://Datasets//Multiagent//rtb')
Datasets_features = {}
for Filename in glob('Validation.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Validation = Datasets_features['Validation']

Wall time: 4.49 s


#### Bid strategies
Define the different explored bid strategies

In [3]:
# Define Bid Strategies

def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))

def ortb_bidding(pCTR, c=50, lmda=5.2e-7, ortbtype=2):
    if ortbtype==2: 
        expr = (pCTR + np.sqrt(c ** 2 * lmda ** 2 + pCTR **2)) / (c * lmda)
        return c * (expr ** (1 / 3) - (1 / expr) ** (1 / 3))  
    else:
        return np.sqrt(c  / lmda * pCTR + c ** 2) - c



#### Define Agent and Agents classes

In [96]:
class Agent:
    
    initPCR = 0.0007375623256   # PCR from Train 
    initBudget = 6250           # Initial budget
        
    def __init__(self, id, pCTR_strategy, bid_strategy, params, superagent = False, factors = [1,2,20]):
        self.budget = self.initBudget
        self.id = id
        self.pCTR_strategy = pCTR_strategy
        self.bid_strategy = bid_strategy
        self.params = params        
        self.pCTRs = list(Datasets_CTR[self.pCTR_strategy +'_validation_pCTR']['pCTR'].values)
        self.bids = tuple()
        self.superagent = superagent
        self.initBids(factors)
            
    def initBids(self,factors):
        bids = []
        for pCTR in self.pCTRs:
            if(self.bid_strategy == "Linear"):
                bids.append(linear_bidding(self.params[0],pCTR,self.initPCR))
            elif(self.bid_strategy == "NLogN"):
                bids.append(nlogn_bidding(self.params[0],pCTR,self.initPCR)) 
            elif(self.bid_strategy == "Exp"):
                bids.append(exp_bidding(self.params[0],pCTR,self.initPCR))
            elif(self.bid_strategy == "ORTB"):
                bids.append(ortb_bidding(pCTR,self.params[0],self.params[1],self.params[2]))
            else:
                raise("Bid strategy not found for agent ", self.id)
        self.bids = bids
        if(self.superagent):
            # Super agents have a multiplying factor for bins for its pCTRs.
            # Higher pCTRs, higher improved bid price 
            factor = np.linspace(factors[0],factors[1],factors[2])
            df = pd.DataFrame.from_records([self.pCTRs,self.bids]).T
            df.rename(columns={0:'pCTR',1:'originalbid'},inplace=True)
            #display(df)
            df['factor'] = pd.cut(df['pCTR'],factors[2], labels=factor)
            df['bidprice'] = (df['originalbid'] * np.single(df['factor']))
            df.bidprice = df.bidprice.round(1)
            self.bids = list(df.bidprice)
        return
        
     
    # Update budget based on bid, returns True if budget allows
    def updateBudget(self,bid,overbid=False):
        if(not overbid):
            if (self.budget - bid) < 0:
                print("ERROR: not enough budget for bid")
                return False
            else:
                self.budget = self.budget - bid
                return True
        else:
            self.budget = self.budget - bid
            return

        
    def canBid(self,bid):
        if(self.budget>bid):
            return True
        return False
    
    #Ensure that bid is within budget
    def getMaxBid(self,idx,overbid=False):
        if (overbid and self.budget>0) or (self.budget > self.bids[idx]):
            return self.bids[idx]
        else:
            return self.budget
    
    def addBids(self,bids):
        self.bids = bids
        return
    
class AgentsClass:    
   
    def __init__(self):
       self.Agents = []
       return
   
    def addA(self,agent):
       self.Agents.append(agent)
   
    def get(self):
       return self.Agents
   
    

#### Define general auction functions

In [58]:

def getBidsCurrentAuction(agents, idx, overbid = False, verbose=False):
    bidsCurrentAuction = []
    for agent in agents:        
        if(verbose):
            print('Bid[',agent.id,']:', agent.getMaxBid(idx,overbid))
        bidsCurrentAuction.append([agent, agent.getMaxBid(idx,overbid)])
    return pd.DataFrame(bidsCurrentAuction, columns=["Agent","Bid"])

def secondPriceWinner(AgentsBidsDf, floorPrice, idx=0, overbid=False,verbose=False):
    #if(idx> 288398 ):
    #    verbose=True
    bestBids= AgentsBidsDf.sort_values(by="Bid", ascending=False)
    if(verbose):
        print(bestBids)
    highestBidAgent = bestBids.iloc[0,0]
    curr_bid = bestBids.iloc[0,1]
    secondPriceOthers = bestBids.iloc[1,1]
    #print(curr_bid, floorPrice)
    if(curr_bid >= floorPrice):
        actualPayPrice = max(floorPrice,secondPriceOthers)
        highestBidAgent.updateBudget(actualPayPrice/1000, overbid)
        if (verbose):
            print(idx,"\tWon: ", highestBidAgent.id, " Paid:" , '{:.3f}'.format(floorPrice), " Upd budget:", '{:.3f}'.format(highestBidAgent.budget))
        return highestBidAgent
    else:        
        if(verbose):
            print(idx, "\tNo winner, Floor:",floorPrice)
        return None
  
def individualAuctions(verbose=False):
    overbid = False
    results_1 = {}
    for idx in tnrange(len(Validation), desc='Simulating auction'):
    #for idx in tnrange(2002, desc='Simulating auction'):    
        #curr_bid = Validation.iloc[idx]
        #if(verbose):
        #    print (Validation.iloc[idx])
        df = getBidsCurrentAuction(agentsList.get(),idx,overbid,verbose=False)
        floorPrice = Validation['payprice'][idx]
        winner = secondPriceWinner(df,floorPrice,idx,overbid,verbose=verbose)
        if (winner != None):
            results_1[idx]=(idx,winner.id,winner.budget)            
        else:
            results_1[idx]=(idx,np.NaN,np.NaN)
        
    results_df=pd.DataFrame.from_dict(results_1).transpose()
    return results_df  

#### Auction #1 (including Exp & NLogN bid strat)
This includes exp and nlogn bid strategies. Next one does not.

In [59]:

agentsList = AgentsClass()

#Create  agents
# ML Algs = [LR, LGBM, NN, Ensemble]
# Bid strat = [Linear,Exp,NLogN,ORTB1,ORTB2]
# 2 parameters each =  4 * 5 * 2 = 40
agentsList.addA(Agent(0,  "LR",  "Linear", [1])) # "Naive" agent

# LR Agents
agentsList.addA(Agent(1,  "LR",  "Linear", [95]))
agentsList.addA(Agent(2,  "LR",  "Linear", [100]))
agentsList.addA(Agent(3,  "LR",  "Exp",    [30]))
agentsList.addA(Agent(4,  "LR",  "Exp",    [27]))
agentsList.addA(Agent(5,  "LR",  "NLogN",  [5000]))
agentsList.addA(Agent(6,  "LR",  "NLogN",  [7500]))
agentsList.addA(Agent(7,  "LR",  "ORTB",   [339.28,3.41e-6,1]))
agentsList.addA(Agent(8,  "LR",  "ORTB",   [350,3.41e-6,1]))
agentsList.addA(Agent(9,  "LR",  "ORTB",   [133.34,4.19e-6,2]))
agentsList.addA(Agent(10, "LR",  "ORTB",   [122.223,4.12e-6,2]))

#LGBM Agents
agentsList.addA(Agent(11,  "LGBM",  "Linear",  [106.1]))
agentsList.addA(Agent(12,  "LGBM",  "Linear",  [106.2]))
agentsList.addA(Agent(13,  "LGBM",  "Exp",     [30]))
agentsList.addA(Agent(14,  "LGBM",  "Exp",     [35]))
agentsList.addA(Agent(15,  "LGBM",  "NLogN",   [5623.4]))
agentsList.addA(Agent(16,  "LGBM",  "NLogN",   [7500]))
agentsList.addA(Agent(17,  "LGBM",  "ORTB",    [299.09,2.84e-6,1]))
agentsList.addA(Agent(18,  "LGBM",  "ORTB",    [314.545,2.84e-6,1]))
agentsList.addA(Agent(19,  "LGBM",  "ORTB",    [221.81,4.1e-6,2]))
agentsList.addA(Agent(20,  "LGBM",  "ORTB",    [237.27,4.1e-6,2]))

#NN Agents
agentsList.addA(Agent(21,  "NN",  "Linear",  [315]))
agentsList.addA(Agent(22,  "NN",  "Linear",  [316]))
agentsList.addA(Agent(23,  "NN",  "Exp",     [45]))
agentsList.addA(Agent(24,  "NN",  "Exp",     [50]))
agentsList.addA(Agent(25,  "NN",  "NLogN",   [11450]))
agentsList.addA(Agent(26,  "NN",  "NLogN",   [15000]))
agentsList.addA(Agent(27,  "NN",  "ORTB",    [144.44,7.71e-7,1]))
agentsList.addA(Agent(28,  "NN",  "ORTB",    [133.33,7.71e-7,1]))
agentsList.addA(Agent(29,  "NN",  "ORTB",    [122.22,1.04e-6,2]))
agentsList.addA(Agent(30,  "NN",  "ORTB",    [100,1.04e-6,2]))

#NN Agents
agentsList.addA(Agent(31,  "ensemble_wavg",  "Linear",  [105.5]))
agentsList.addA(Agent(32,  "ensemble_wavg",  "Linear",  [106]))
agentsList.addA(Agent(33,  "ensemble_wavg",  "Exp",     [33.8]))
agentsList.addA(Agent(34,  "ensemble_wavg",  "Exp",     [32.4]))
agentsList.addA(Agent(35,  "ensemble_wavg",  "NLogN",   [1968]))
agentsList.addA(Agent(36,  "ensemble_wavg",  "NLogN",   [2411]))
agentsList.addA(Agent(37,  "ensemble_wavg",  "ORTB",    [186.66,2.89e-6,1]))
agentsList.addA(Agent(38,  "ensemble_wavg",  "ORTB",    [200,2.89e-6,1]))
agentsList.addA(Agent(39,  "ensemble_wavg",  "ORTB",    [240,4.28e-6,2]))
agentsList.addA(Agent(40,  "ensemble_wavg",  "ORTB",    [208.88,4.07e-6,2]))


debug = True


df = individualAuctions(False)
df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'},inplace=True)
df.head()

results ={}
for idx in range(len(agentsList.get())):    
    agent = agentsList.get()[idx]    
    a_idx_won = df[df["Winner"] == agent.id]['Ad']    
    a_won = Validation.iloc[a_idx_won]
    a_clicks = a_won[a_won['click'] == 1]['click'].sum()
    a_imps = len(a_won)
    a_eCTR = a_clicks/max(0.1,a_imps)
    a_spent = -1 * (agent.budget-Agent.initBudget)
    a_avg_CPM = a_spent/max(0.1,a_imps)
    a_avg_CPC = 0
    if(a_clicks > 0):
        a_avg_CPC = a_spent/a_clicks
    results[idx] = ('Agent'+str(agent.id),agent.pCTR_strategy,agent.bid_strategy,agent.params,a_clicks,a_imps,round(a_spent,3),a_eCTR,a_avg_CPM,a_avg_CPC)
#results
results_df = pd.DataFrame.from_dict(results).transpose()
results_df.rename(columns={0:'Agent', 1:'pCTR', 2:'Bidding',3:'BB', 4:'Clicks', 5:'Imps' , 6:'Spent', 7:'eCTR', 8:'avgCPM', 9:'avgCPC'}, inplace=True)
print('Total clicks: ', results_df['Clicks'].sum())
display(results_df.sort_values(['Clicks','eCTR','Imps'], ascending=False))

C:\Users\mau_c\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


,Agent,pCTR,Bidding,BB,Clicks,Imps,Spent,eCTR,avgCPM,avgCPC
28,Agent28,NN,ORTB,"[133.33, 7.71e-07, 1]",20,53,322.313,0.377358,6.08138,16.1156
27,Agent27,NN,ORTB,"[144.44, 7.71e-07, 1]",16,109,403.164,0.146789,3.69876,25.1978
21,Agent21,NN,Linear,[315],16,5862,5376.08,0.00272944,0.917106,336.005
31,Agent31,ensemble_wavg,Linear,[105.5],15,52,316.564,0.288462,6.08777,21.1043
32,Agent32,ensemble_wavg,Linear,[106],9,426,339.692,0.0211268,0.797399,37.7436
23,Agent23,NN,Exp,[45],9,1814,5224.53,0.00496141,2.88012,580.504
3,Agent3,LR,Exp,[30],9,7123,5744.97,0.00126351,0.806538,638.33
34,Agent34,ensemble_wavg,Exp,[32.4],8,1805,5221.83,0.00443213,2.89298,652.729
13,Agent13,LGBM,Exp,[30],8,1886,5221.68,0.00424178,2.76865,652.71
22,Agent22,NN,Linear,[316],8,17387,6096.5,0.000460114,0.350635,762.062


#### Auction #2,  only with reported bidding strategies

In [61]:


agentsList = AgentsClass()

#Create 
#ML Algs[LR, LGBM, NN, Ensemble]
#Bid strat [Linear,Exp,NLogN,ORTB1,ORTB2]
#2 parameters each =  4 * 5 * 2 = 40
agentsList.addA(Agent(0,  "LR",  "Linear", [1])) # "Naive" agent

# LR Agents
agentsList.addA(Agent(1,  "LR",  "Linear", [95]))
agentsList.addA(Agent(2,  "LR",  "Linear", [100]))
agentsList.addA(Agent(3,  "LR",  "ORTB",   [339.28,3.41e-6,1]))
agentsList.addA(Agent(4,  "LR",  "ORTB",   [350,3.41e-6,1]))
agentsList.addA(Agent(5,  "LR",  "ORTB",   [133.34,4.19e-6,2]))
agentsList.addA(Agent(6, "LR",  "ORTB",   [122.223,4.12e-6,2]))

#LGBM Agents
agentsList.addA(Agent(7,  "LGBM",  "Linear",  [106.1]))
agentsList.addA(Agent(8,  "LGBM",  "Linear",  [106.2]))
agentsList.addA(Agent(9,  "LGBM",  "ORTB",    [299.09,2.84e-6,1]))
agentsList.addA(Agent(10,  "LGBM",  "ORTB",    [314.545,2.84e-6,1]))
agentsList.addA(Agent(11,  "LGBM",  "ORTB",    [221.81,4.1e-6,2]))
agentsList.addA(Agent(12,  "LGBM",  "ORTB",    [237.27,4.1e-6,2]))

#NN Agents
agentsList.addA(Agent(13,  "NN",  "Linear",  [315]))
agentsList.addA(Agent(14,  "NN",  "Linear",  [316]))
agentsList.addA(Agent(15,  "NN",  "ORTB",    [144.44,7.71e-7,1]))
agentsList.addA(Agent(16,  "NN",  "ORTB",    [133.33,7.71e-7,1]))
agentsList.addA(Agent(17,  "NN",  "ORTB",    [122.22,1.04e-6,2]))
agentsList.addA(Agent(18,  "NN",  "ORTB",    [100,1.04e-6,2]))

#NN Agents
agentsList.addA(Agent(19,  "ensemble_wavg",  "Linear",  [105.5]))
agentsList.addA(Agent(20,  "ensemble_wavg",  "Linear",  [106]))
agentsList.addA(Agent(21,  "ensemble_wavg",  "ORTB",    [186.66,2.89e-6,1]))
agentsList.addA(Agent(22,  "ensemble_wavg",  "ORTB",    [200,2.89e-6,1]))
agentsList.addA(Agent(23,  "ensemble_wavg",  "ORTB",    [240,4.28e-6,2]))
agentsList.addA(Agent(24,  "ensemble_wavg",  "ORTB",    [208.88,4.07e-6,2]))


df = individualAuctions(False)
df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'},inplace=True)
df.head()

results ={}
for idx in range(len(agentsList.get())):    
    agent = agentsList.get()[idx]    
    a_idx_won = df[df["Winner"] == agent.id]['Ad']    
    a_won = Validation.iloc[a_idx_won]
    a_clicks = a_won[a_won['click'] == 1]['click'].sum()
    a_imps = len(a_won)
    a_eCTR = a_clicks/max(0.1,a_imps)
    a_spent = -1 * (agent.budget-Agent.initBudget)
    a_avg_CPM = a_spent/max(0.1,a_imps)
    a_avg_CPC = 0
    if(a_clicks > 0):
        a_avg_CPC = a_spent/a_clicks
    results[idx] = ('Agent'+str(agent.id),agent.pCTR_strategy,agent.bid_strategy,agent.params,a_clicks,a_imps,round(a_spent,3),a_eCTR,a_avg_CPM,a_avg_CPC)
#results
results_df = pd.DataFrame.from_dict(results).transpose()
results_df.rename(columns={0:'Agent', 1:'pCTR', 2:'Bidding',3:'BB', 4:'Clicks', 5:'Imps' , 6:'Spent', 7:'eCTR', 8:'avgCPM', 9:'avgCPC'}, inplace=True)

print('Total clicks: ', results_df['Clicks'].sum())
display(results_df.sort_values(['Clicks','eCTR','Imps'], ascending=False))



Total clicks:  180


,Agent,pCTR,Bidding,BB,Clicks,Imps,Spent,eCTR,avgCPM,avgCPC
16,Agent16,NN,ORTB,"[133.33, 7.71e-07, 1]",33,178,529.374,0.185393,2.97401,16.0416
15,Agent15,NN,ORTB,"[144.44, 7.71e-07, 1]",29,14811,5568.9,0.001958,0.375998,192.031
13,Agent13,NN,Linear,[315],22,12976,6100.27,0.00169544,0.470119,277.285
2,Agent2,LR,Linear,[100],18,25818,6153.55,0.000697188,0.238343,341.864
19,Agent19,ensemble_wavg,Linear,[105.5],16,76,438.325,0.210526,5.76744,27.3953
8,Agent8,LGBM,Linear,[106.2],15,12871,3849.28,0.00116541,0.299066,256.618
14,Agent14,NN,Linear,[316],13,12375,6100.91,0.00105051,0.493003,469.301
12,Agent12,LGBM,ORTB,"[237.27, 4.1e-06, 2]",9,25529,2628.26,0.00035254,0.102952,292.029
1,Agent1,LR,Linear,[95],8,4882,2182,0.00163867,0.446949,272.75
20,Agent20,ensemble_wavg,Linear,[106],5,3521,1212.19,0.00142005,0.344273,242.437


#### Auction playzone, to test superagents
A super-agent is defined to try to beat all other agents. This super-agent has boosted bid prices.

In [94]:

#With NN Linear superagent

agentsList = AgentsClass()

#Create 
#ML Algs[LR, LGBM, NN, Ensemble]
#Bid strat [Linear,Exp,NLogN,ORTB1,ORTB2]
#2 parameters each =  4 * 5 * 2 = 40
agentsList.addA(Agent(0,  "LR",  "Linear", [1])) # "Naive" agent

# LR Agents
agentsList.addA(Agent(1,  "LR",  "Linear", [95]))
agentsList.addA(Agent(2,  "LR",  "Linear", [100]))
agentsList.addA(Agent(3,  "LR",  "ORTB",   [339.28,3.41e-6,1]))
agentsList.addA(Agent(4,  "LR",  "ORTB",   [350,3.41e-6,1]))
agentsList.addA(Agent(5,  "LR",  "ORTB",   [133.34,4.19e-6,2]))
agentsList.addA(Agent(6, "LR",  "ORTB",   [122.223,4.12e-6,2]))

#LGBM Agents
agentsList.addA(Agent(7,   "LGBM",  "Linear",  [106.1]))
agentsList.addA(Agent(8,   "LGBM",  "Linear",  [106.2]))
agentsList.addA(Agent(9,   "LGBM",  "ORTB",    [299.09,2.84e-6,1]))
agentsList.addA(Agent(10,  "LGBM",  "ORTB",    [314.545,2.84e-6,1]))
agentsList.addA(Agent(11,  "LGBM",  "ORTB",    [221.81,4.1e-6,2]))
agentsList.addA(Agent(12,  "LGBM",  "ORTB",    [237.27,4.1e-6,2]))

#NN Agents
agentsList.addA(Agent(13,  "NN",  "Linear",  [315]))
agentsList.addA(Agent(14,  "NN",  "Linear",  [316]))
agentsList.addA(Agent(15,  "NN",  "ORTB",    [144.44,7.71e-7,1]))
agentsList.addA(Agent(16,  "NN",  "ORTB",    [133.33,7.71e-7,1]))
agentsList.addA(Agent(17,  "NN",  "ORTB",    [122.22,1.04e-6,2]))
agentsList.addA(Agent(18,  "NN",  "ORTB",    [100,1.04e-6,2]))

#NN Agents
agentsList.addA(Agent(19,  "ensemble_wavg",  "Linear",  [105.5]))
agentsList.addA(Agent(20,  "ensemble_wavg",  "Linear",  [106]))
agentsList.addA(Agent(21,  "ensemble_wavg",  "ORTB",    [186.66,2.89e-6,1]))
agentsList.addA(Agent(22,  "ensemble_wavg",  "ORTB",    [200,2.89e-6,1]))
agentsList.addA(Agent(23,  "ensemble_wavg",  "ORTB",    [240,4.28e-6,2]))
agentsList.addA(Agent(24,  "ensemble_wavg",  "ORTB",    [208.88,4.07e-6,2]))

#Super-Agent
agentsList.addA(Agent(25,"NN",  "Linear", [315] , True , [1.5,3.5,20]))

df = individualAuctions(False)
df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'},inplace=True)
df.head()

results ={}
for idx in range(len(agentsList.get())):    
    agent = agentsList.get()[idx]    
    a_idx_won = df[df["Winner"] == agent.id]['Ad']    
    a_won = Validation.iloc[a_idx_won]
    a_clicks = a_won[a_won['click'] == 1]['click'].sum()
    a_imps = len(a_won)
    a_eCTR = a_clicks/max(0.1,a_imps)
    a_spent = -1 * (agent.budget-Agent.initBudget)
    a_avg_CPM = a_spent/max(0.1,a_imps)
    a_avg_CPC = 0
    if(a_clicks > 0):
        a_avg_CPC = a_spent/a_clicks
    results[idx] = ('Agent'+str(agent.id),agent.pCTR_strategy,agent.bid_strategy,agent.params,a_clicks,a_imps,round(a_spent,3),a_eCTR,a_avg_CPM,a_avg_CPC)
#results
results_df = pd.DataFrame.from_dict(results).transpose()
results_df.rename(columns={0:'Agent', 1:'pCTR', 2:'Bidding',3:'BB', 4:'Clicks', 5:'Imps' , 6:'Spent', 7:'eCTR', 8:'avgCPM', 9:'avgCPC'}, inplace=True)

print('Total clicks: ', results_df['Clicks'].sum())
display(results_df.sort_values(['Clicks','eCTR','Imps'], ascending=False))

# Play sound when finished
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


Total clicks:  180


,Agent,pCTR,Bidding,BB,Clicks,Imps,Spent,eCTR,avgCPM,avgCPC
16,Agent16,NN,ORTB,"[133.33, 7.71e-07, 1]",31,74,403.577,0.418919,5.45374,13.0186
15,Agent15,NN,ORTB,"[144.44, 7.71e-07, 1]",26,9236,3811.13,0.00281507,0.412639,146.582
13,Agent13,NN,Linear,[315],19,12137,6100.13,0.00156546,0.502606,321.06
2,Agent2,LR,Linear,[100],17,26399,6152.68,0.000643964,0.233065,361.922
14,Agent14,NN,Linear,[316],14,10493,6100.96,0.00133422,0.581431,435.783
8,Agent8,LGBM,Linear,[106.2],14,12087,3559.16,0.00115827,0.294461,254.225
19,Agent19,ensemble_wavg,Linear,[105.5],13,72,417.903,0.180556,5.80421,32.1464
25,Agent25,NN,Linear,[315],12,20740,6212.09,0.000578592,0.299522,517.674
12,Agent12,LGBM,ORTB,"[237.27, 4.1e-06, 2]",9,24971,2566.19,0.000360418,0.102767,285.132
1,Agent1,LR,Linear,[95],8,3489,1832.1,0.00229292,0.525107,229.012


##### Performance super agents
Different outcomes from super-agents given the defined parameters (Alg, bid, num of bins, factor, etc)

| ID | Alg | Bid | Params | Clicks | Imps | Spent | Comments |
| - | - | - | - | - | - | - | - |
| 25 | "LGBM" |"ORTB" | [314.545,2.84e-6,1], True, [1,2] | 0 | 14996 | 743.359|  |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1,2] | 15 | 108 | 407.499 | Other NN ORTB were first, stole from both |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.3,2.5] | 21 | 56159 | 6230.17 | 4th place (33,30,21,21) |
| 25 | "NN" | "ORTB" | [133.33, 7.71e-07, 1] , True , [1.3,2.5] | 10	| 40173 | 6204.51 |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.5,3] | 25 | 55933 | 6232.25 | 3rd (33,29,25) after NN ORTBs |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.1,3] | 20| 39044 | 6178.66 | 4th (33,30,21,20) |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.2,3.5] | 22 | 55519 | 6227.96 | 3rd(33,30,22) after NN ORTBs |
| 25 | "NN" | "Linear" | [315], True, [1.5,3] | 12 | 20740 | 6212.09 | 8th |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1.1,3] | 19 | 32068 | 6165.86 | 4th, 29,23,20,19 |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1.05,3.5] | 25 | 20268 | 6036.89 | 2nd (28,25) #1 got 28clkcs/61 imps |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1.05,3.2] | 25 | 20281 | 6036.76 | 2nd (28,25)  |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1.01,3] | 15 | 149 | 462.723 | 5th 27,26,19,18,15  stole 2 26/59  |
| 25 | "NN" |"ORTB" | [133.33, 7.71e-07, 1], True, [1.05,3] | 24 | 20287 | 6036.44 | 2nd (28,24) 28/61  |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.05,3.5] | 16 | 21984 | 5149.74 | 5th (32,30,22,18,16) |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.1,3.5] | 20 | 39026 | 6178.75 | 5th (33,30,21,20) |
| 25 | "LGBM" | "Linear" | [106.2], True, [2,3] | 24 | 53271 | 6234.16 | 3rd 32,30,24 |
| 25 | "LR" | "Linear" | [95], True, [1.1,3,20] | 17 | 28703 | 6169.04 | 4th | 

With X bins

| ID | Alg | Bid | Params | Clicks | Imps | Spent | Comments |
| - | - | - | - | - | - | - | - |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.5,3,10] | 5 | 6558  | 1981.85  |  |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.5,3,30] | 25 | 55852  | 6232.26  | 3rd, 33, 29 |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.3,4,30] | 21 | 56004  | 6230.23  |  |
| 25 | "LGBM" | "Linear" | [106.2], True, [1.3,4,40] | 25 | 55697 | 6232.25  |  | 




In [ ]:
#could test
np.flip(1-np.logspace(0.0,1,20)/10) + 1.1